
# LSTM no contexto de clientes do BuonoPreço
**Previsão de `gasto_medio` do próximo mês a partir de uma janela temporal curta.**

Este notebook:
1) Lê o dataset `dataset-buonopreco-registro_de_clientes.csv` do repositório **AI-Lab** (caminho local `AI-Lab/datasets`) ou diretamente do **GitHub (raw URL)**.  
2) Cria **séries temporais sintéticas** por cliente (12 meses) a partir dos atributos estáticos do dataset.  
3) Constrói janelas deslizantes `T`→`t+1` e treina uma **LSTM** em PyTorch para prever `gasto_medio` no próximo passo.  
4) Plota curvas de loss e uma comparação `y verdadeiro` vs `y predito` no conjunto de validação.

> **Nota:** o dataset original é estático (um registro por cliente). Para fins didáticos, geramos **trajetórias mensais plausíveis** baseadas nesses perfis, preservando *ordens de grandeza* e relações entre variáveis (frequência, desconto, WhatsApp etc.).



## 1. Imports e Configurações


In [ ]:

import os
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Reprodutibilidade básica
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Matplotlib sem estilos/cor explícita
plt.rcParams.update({"figure.figsize": (7,4)})



## 2. Leitura do dataset

Tentamos os seguintes caminhos em ordem:
1. **Local** (caso o repositório esteja clonado): `../datasets/dataset-buonopreco-registro_de_clientes.csv` e `../../datasets/...`
2. **Raw GitHub**: `https://raw.githubusercontent.com/sousamaf/AI-Lab/main/datasets/...` (ou `master`)

> Em **Google Colab**, esta célula funciona diretamente, pois há acesso à internet. Alternativamente, você pode montar o Google Drive ou clonar o repositório.


In [ ]:

CANDIDATES = [
    './AI-Lab/datasets/dataset-buonopreco-registro_de_clientes.csv',
    '../AI-Lab/datasets/dataset-buonopreco-registro_de_clientes.csv',
    '../../AI-Lab/datasets/dataset-buonopreco-registro_de_clientes.csv',
    '../datasets/dataset-buonopreco-registro_de_clientes.csv',
    '../../datasets/dataset-buonopreco-registro_de_clientes.csv',
    'https://raw.githubusercontent.com/sousamaf/AI-Lab/main/datasets/dataset-buonopreco-registro_de_clientes.csv',
    'https://raw.githubusercontent.com/sousamaf/AI-Lab/master/datasets/dataset-buonopreco-registro_de_clientes.csv',
]

last_err = None
df = None
for path in CANDIDATES:
    try:
        df = pd.read_csv(path)
        print('Carregado de:', path)
        break
    except Exception as e:
        last_err = e

if df is None:
    raise RuntimeError(f'Não foi possível carregar o dataset. Último erro: {last_err}')

df.head()



## 3. Preparação e limpeza

- Conversão de tipos numéricos
- Criação de índices categóricos para `categoria_preferida` e `classe_cliente` (para uso opcional)  
- Checagem da coluna `taxa_resposta_whatsapp` (já fornecida no dataset de exemplo)


In [ ]:

num_cols = ['idade','frequencia_mensal','gasto_medio','pct_compras_com_desconto',
            'whatsapp_envios','whatsapp_respostas','taxa_resposta_whatsapp']

for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

df['categoria_preferida'] = df['categoria_preferida'].astype('category')
df['classe_cliente'] = df['classe_cliente'].astype('category')

cat2idx = dict(enumerate(df['categoria_preferida'].cat.categories))
cat_map = {v:k for k,v in cat2idx.items()}
df['categoria_idx'] = df['categoria_preferida'].map(cat_map)

class2idx = dict(enumerate(df['classe_cliente'].cat.categories))
class_map = {v:k for k,v in class2idx.items()}
df['classe_idx'] = df['classe_cliente'].map(class_map)

df[num_cols].describe()



## 4. Geração de **séries temporais sintéticas** por cliente (12 meses)

Usamos um **modelo simples** para simular a evolução mensal de variáveis:
- `frequencia_mensal_t` segue um **random walk suave** em torno da base do cliente.
- `pct_compras_com_desconto_t` e `taxa_resposta_whatsapp_t` oscilam levemente (sazonalidade curta + ruído).
- `gasto_medio_t+1` é **autoregressivo** com dependência em `frequencia`, `desconto` e `resposta_whatsapp`.

> É um **modelo didático**, não prescritivo. Parâmetros foram escolhidos para manter valores plausíveis.


In [ ]:

def simulate_customer_series(row, T=12):
    base_freq = float(row['frequencia_mensal'])
    base_spend = float(row['gasto_medio'])
    base_disc = float(row['pct_compras_com_desconto']) / 100.0
    base_rr = float(row['taxa_resposta_whatsapp'])  # 0..1
    cat_idx = int(row['categoria_idx'])
    idade = float(row['idade'])

    freq, disc, rr, spend = [], [], [], []
    f, d, r, s = base_freq, base_disc, base_rr, base_spend

    for t in range(T):
        f = 0.8*f + 0.2*base_freq + np.random.normal(0, 0.03)
        d = np.clip(0.8*d + 0.2*base_disc + np.random.normal(0, 0.01), 0, 1)
        r = np.clip(0.85*r + 0.15*base_rr + 0.05*np.sin(2*np.pi*t/6) + np.random.normal(0, 0.02), 0, 1)
        s = (0.65*s + 30.0*f + 80.0*r - 20.0*d + 0.05*idade + (cat_idx % 3)*5.0 + np.random.normal(0, 10.0))
        s = max(10.0, s)

        freq.append(f)
        disc.append(d*100.0)
        rr.append(r)
        spend.append(s)

    return pd.DataFrame({
        'frequencia_mensal': freq,
        'pct_compras_com_desconto': disc,
        'taxa_resposta_whatsapp': rr,
        'gasto_medio': spend,
        'categoria_idx': cat_idx,
        'idade': idade,
        'cliente_id': row['cliente_id']
    })

all_series = [simulate_customer_series(row, T=12) for _, row in df.iterrows()]
ts_df = pd.concat(all_series, axis=0, ignore_index=True)
ts_df.head()



## 5. Janelas deslizantes e *dataset* PyTorch

Definimos janela `WINDOW=6` para prever `gasto_medio` no passo `t+1`.
Entradas: `[frequencia, desconto, resposta, idade, categoria_embedding]`.


In [ ]:

from torch.utils.data import Dataset, DataLoader
WINDOW = 6

feature_cols_numeric = ['frequencia_mensal','pct_compras_com_desconto','taxa_resposta_whatsapp','idade']
cat_col = 'categoria_idx'
target_col = 'gasto_medio'

num_features = len(feature_cols_numeric)

groups = ts_df.groupby('cliente_id')
X_num, X_cat, y = [], [], []
for cid, g in groups:
    g = g.reset_index(drop=True)
    for t in range(len(g) - WINDOW):
        X_num.append(g.loc[t:t+WINDOW-1, feature_cols_numeric].values.astype('float32'))
        X_cat.append(g.loc[t:t+WINDOW-1, cat_col].values.astype('int64'))
        y.append(g.loc[t+WINDOW, target_col].astype('float32'))

X_num = np.array(X_num)
X_cat = np.array(X_cat)
y = np.array(y).reshape(-1, 1)

N = len(X_num)
idx = np.arange(N)
np.random.shuffle(idx)
split = int(0.8*N)
train_idx, val_idx = idx[:split], idx[split:]

Xn_tr, Xc_tr, y_tr = X_num[train_idx], X_cat[train_idx], y[train_idx]
Xn_va, Xc_va, y_va = X_num[val_idx], X_cat[val_idx], y[val_idx]

class SeqDataset(Dataset):
    def __init__(self, Xn, Xc, y):
        self.Xn = torch.from_numpy(Xn)
        self.Xc = torch.from_numpy(Xc)
        self.y  = torch.from_numpy(y)
    def __len__(self): return self.Xn.shape[0]
    def __getitem__(self, i):
        return self.Xn[i], self.Xc[i], self.y[i]

train_ds = SeqDataset(Xn_tr, Xc_tr, y_tr)
val_ds   = SeqDataset(Xn_va, Xc_va, y_va)

train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
val_dl   = DataLoader(val_ds, batch_size=128, shuffle=False)

n_categories = int(df['categoria_idx'].max()) + 1
n_categories



## 6. Modelo LSTM (com *embedding* para categoria)


In [ ]:

class LSTMBuono(nn.Module):
    def __init__(self, num_features, n_categories, emb_dim=4, hidden=32, layers=1, dropout=0.0):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings=n_categories, embedding_dim=emb_dim)
        self.lstm = nn.LSTM(input_size=num_features + emb_dim, hidden_size=hidden, num_layers=layers, dropout=dropout if layers>1 else 0.0, batch_first=True)
        self.fc = nn.Linear(hidden, 1)

    def forward(self, x_num, x_cat):
        emb = self.emb(x_cat)
        x = torch.cat([x_num, emb], dim=-1)
        out, (h, c) = self.lstm(x)
        return self.fc(out[:, -1, :])

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu")

model = LSTMBuono(num_features=num_features, n_categories=n_categories, emb_dim=4, hidden=32).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

def run_epoch(dloader, train=True):
    model.train() if train else model.eval()
    total, count = 0.0, 0
    for xb_num, xb_cat, yb in dloader:
        xb_num, xb_cat, yb = xb_num.to(device), xb_cat.to(device), yb.to(device)
        with torch.set_grad_enabled(train):
            yhat = model(xb_num, xb_cat)
            loss = loss_fn(yhat, yb)
            if train:
                opt.zero_grad()
                loss.backward()
                opt.step()
        total += float(loss.item()) * xb_num.size(0)
        count += xb_num.size(0)
    return total / max(1, count)

train_losses, val_losses = [], []
EPOCHS = 25
for ep in range(EPOCHS):
    tl = run_epoch(train_dl, True)
    vl = run_epoch(val_dl, False)
    train_losses.append(tl); val_losses.append(vl)
    if (ep+1) % 5 == 0 or ep == 0:
        print(f"Época {ep+1:02d} | treino MSE: {tl:.2f} | val MSE: {vl:.2f}")


In [ ]:

plt.figure()
plt.plot(train_losses, label='treino')
plt.plot(val_losses, label='validação')
plt.xlabel('época'); plt.ylabel('MSE'); plt.title('Curvas de Loss'); plt.legend(); plt.show()



## 7. Avaliação simples no conjunto de validação


In [ ]:

model.eval()
truth, pred = [], []
with torch.no_grad():
    for xb_num, xb_cat, yb in val_dl:
        yhat = model(xb_num.to(device), xb_cat.to(device)).cpu().numpy().ravel()
        pred.extend(yhat.tolist())
        truth.extend(yb.numpy().ravel().tolist())

k = min(120, len(truth))
plt.figure()
plt.plot(truth[:k], label='verdadeiro')
plt.plot(pred[:k], label='predito')
plt.xlabel('amostras'); plt.ylabel('gasto médio'); plt.title('Validação: verdadeiro vs predito'); plt.legend(); plt.show()



## 8. Como usar no **Google Colab**

- Opção A *(recomendada)*: mantenha a estrutura do repositório e **leia via raw GitHub** (já implementado no loader).  
- Opção B: **clonar o repositório**:
```bash
!git clone https://github.com/sousamaf/AI-Lab.git
%cd AI-Lab/algorithms/neural_networks/lstm
```
- Opção C: **montar o Google Drive** e apontar `CANDIDATES` para o caminho no Drive.

Este notebook foi construído para funcionar mesmo fora do repositório, desde que o **raw GitHub** esteja acessível.
